In [ ]:
import sys
home_dir = '/Users/an88/'
sys.path.insert(0, home_dir + 'Dropbox/Documents (Academic)/GitHub/NUTS (Python)/')
sys.path.insert(0, home_dir + 'Dropbox/Documents (Academic)/GitHub/Discontinuous HMC/')
sys.path.insert(0, home_dir + 'Dropbox/Documents (Academic)/GitHub/Discontinuous HMC/dhmc/')
from importlib import reload
from dhmc_sampler import DHMCSampler
from NUTS import nuts
from adaptive_metropolis import adap_RWMH, RWMH
from mcmc_diagnostic import *

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle as pkl
import warnings
import pdb
import time
%matplotlib inline

### Import the Jolly-Seber model with the black-kneed capsid data from Seber (1982).

In [ ]:
# Function to compute the log-posterior and its gradient.
from jolly_seber_model import f

# Function to compute the difference in the log conditional density.
from jolly_seber_model import f_update

# The parameters "p", "phi", "U" are concatenated into a 1-d
# array for running DHMC. The dictionary "index" stores the
# linear indices used internally by 'f' and 'f_update'.
from jolly_seber_model import pack_param, unpack_param, index

# Extract the number of continuous and discrete parameters.
n_param = np.sum([len(val) for val in index.values()])
n_disc = len(index["U"])
n_cont = n_param - n_disc

#### Initial state for MCMC.

In [ ]:
phi0 = .8 * np.ones(len(index["phi"]))
p0 = .15 * np.ones(len(index["p"]))
U_init0 = 500
B0 = 200 * np.ones(len(index["U"]) - 1) # The number of "births".
theta0 = pack_param(p0, phi0, U_init0, B0)

#### Test the coordinate wise update function

In [ ]:
scale = np.ones(n_param)
dhmc = DHMCSampler(f, f_update, n_disc, n_param, scale)
dhmc.test_cont_grad(theta0, sd=1, n_test=10);
_, theta, logp_fdiff, logp_diff = \
    dhmc.test_update(theta0, sd=10, n_test=10)

### Define an integrator for discontinuous HMC

In [ ]:
def check_output(samples, logp_samples, accept_prob, nfevals_per_itr, method):
    
    filename = 'jolly_seber_' + method + '_test_output.pkl'
    with open(filename, 'rb') as file:
        samples0, logp_samples0, accept_prob0, nfevals_per_itr0 \
            = pkl.load(file)
            
    test_pass = np.allclose(samples, samples0) \
        and np.allclose(logp_samples, logp_samples0) \
        and np.allclose(accept_prob, accept_prob0) \
        and np.allclose(nfevals_per_itr, nfevals_per_itr0)
        
    if test_pass:
        print('Test passed! The current output matches the former one.')
    else:
        print('Test failed....')

In [ ]:
seed = 1
n_burnin = 10 ** 1
n_sample = 1 * 10 ** 1
n_update = 1
dt = .025 * np.array([.8, 1])
nstep = [70, 85]

samples, logp_samples, accept_prob, nfevals_per_itr, time_elapsed = \
    dhmc.run_sampler(theta0, dt, nstep, n_burnin, n_sample, 
                     seed=seed, n_update=n_update)
samples = samples[n_burnin:, :]
logp_samples = logp_samples[n_burnin:]
check_output(samples, logp_samples, accept_prob, nfevals_per_itr, method='dhmc')

In [ ]:
update_test_output = False
method = 'dhmc'
if update_test_output:
    filename = 'jolly_seber_' + method + '_test_output.pkl'
    with open(filename, 'wb') as file:
        to_save = (samples, logp_samples, accept_prob, nfevals_per_itr)
        pkl.dump(to_save, file)

In [ ]:
mcmc_output = {
    'samples': samples,
    'logp': logp_samples,
    'accept_prob': accept_prob,
    'nfevals_per_itr': nfevals_per_itr,
    'n_burnin': n_burnin,
    'seed': seed,
    'theta0': theta0,
}

In [ ]:
filename = 'jolly_seber_dhmc_output.npy'
with open(filename, 'wb') as file:
    pkl.dump(mcmc_output, file)

### NUTS-Gibbs sampler for comparison.

In [ ]:
from jolly_seber_model import update_disc

In [ ]:
def nuts_gibbs(f, theta, dt, logp, grad, max_depth):
    def f_cond(theta_cont):
        logp, grad, _ = f(np.concatenate((theta_cont, theta[n_cont:])))
        if not np.any(np.isnan(grad)):
            grad = grad[:n_cont]
        return logp, grad
    theta_cont, logp, grad, nuts_accept_prob, nfevals = \
        nuts(f_cond, np.random.uniform(dt[0], dt[1]), theta[:n_cont], logp, grad, max_depth, warnings=False)
    theta[:n_cont] = theta_cont
    theta = update_disc(theta)
    logp, grad, _ = f(theta)
    grad = grad[:n_cont]    
    nfevals += 1
    return theta, logp, grad, nuts_accept_prob, nfevals

In [ ]:
n_burnin = 10 ** 1
n_sample = 1 * 10 ** 1
n_update = 1
seed = 1

np.random.seed(seed)
    
# Pre-allocate
theta = theta0.copy()
n_per_update = math.ceil(n_sample / n_update)
nfevals_total = 0
samples = np.zeros((n_sample + n_burnin, len(theta)))
logp_samples = np.zeros(n_sample + n_burnin)
accept_prob = np.zeros(n_sample + n_burnin)

# Run NUTS-Gibbs
tic = time.process_time()
logp, grad, _ = f(theta)
grad = grad[:n_cont]
for i in range(n_sample + n_burnin):
    theta, logp, grad, accept_prob[i], nfevals = \
        nuts_gibbs(f, theta, dt, logp, grad, max_depth=8)
    nfevals_total += nfevals + 1
    samples[i, :] = theta
    logp_samples[i] = logp
    if (i + 1) % n_per_update == 0:
        print('{:d} iterations have been completed.'.format(i+1))

toc = time.process_time()
time_elapsed = toc - tic
time_elapsed *= n_sample / (n_sample + n_burnin) # Adjust for the burn-in time.  
nfevals_per_itr = nfevals_total / (n_sample + n_burnin)
print('Each iteration required {:.2f} likelihood evaluations on average.'.format(nfevals_per_itr))

samples = samples[n_burnin:, :]
logp_samples = logp_samples[n_burnin:]

check_output(samples, logp_samples, accept_prob, nfevals_per_itr, method='nuts_gibbs')

In [ ]:
update_test_output = False
method = 'nuts_gibbs'
if update_test_output:
    filename = 'jolly_seber_' + method + '_test_output.pkl'
    with open(filename, 'wb') as file:
        to_save = (samples, logp_samples, accept_prob, nfevals_per_itr)
        pkl.dump(to_save, file)

In [ ]:
seed = None
mcmc_output = {
    'samples': samples,
    'logp': logp_samples,
    'accept_prob': accept_prob,
    'nfevals_per_itr': nfevals_per_itr,
    'n_burnin': n_burnin,
    'seed': seed,
    'theta0': theta0,
}

In [ ]:
filename = 'jolly_seber_gibbs_output.npy'
with open(filename, 'wb') as file:
    pkl.dump(mcmc_output, file)

### Try M-H sampler with an optimal proposal variance.

In [ ]:
def f_logp(theta):
    logp, _, _ = f(theta, req_grad=False)
    return logp

n_warmup = 10 ** 3
n_cov_adap = 10 ** 3
n_adap_mcmc = 5 * 10 ** 3
n_sample = 5 * 10 ** 3
seed = 1

np.random.seed(seed)

# Run adaptive MH to estimate the covariance.
stepsize = 2.38 / math.sqrt(n_param)
samples, accept_rate = \
    adap_RWMH(f_logp, theta0, stepsize, n_warmup, n_cov_adap, n_adap_mcmc)
Sigma = np.cov(samples.T)

# Run MH with a fixed covariance.
tic = time.process_time() # Start clock
samples, accept_rate, stepsize_seq, ave_stepsize_seq = \
    RWMH(f_logp, theta0, stepsize, 0, n_sample, Sigma)

toc = time.process_time()
time_elapsed = toc - tic
print('Sampling completed.')

In [ ]:
mcmc_output = {
    'samples': samples,
    'accept_rate': accept_rate,
    'n_warmup': n_warmup,
    'n_cov_adap': n_cov_adap,
    'n_adap_mcmc': n_adap_mcmc,
    'seed': seed,
    'theta0': theta0,
}
filename = 'jolly_seber_mh_output.npy'
with open(filename, 'wb') as file:
    pkl.dump(mcmc_output, file)

### Analyze the MCMC output.

In [ ]:
filename = 'jolly_seber_dhmc_output.pkl'
with open(filename, 'rb') as file:
    mcmc_output = pkl.load(file)
samples = mcmc_output['samples']

In [ ]:
# Frequentist estimates.
phi_hat = np.array([.649, 1.015, .867, .564, .836, .790, .651, .985, .686, .884, .771, float('nan')])
phi_hat_sd = np.array([.114, .110, .107, .064, .075, .070, .056, .093, .080, .120, .128, float('nan')])
N_hat = np.array([float('nan'), 511.2, 779.1, 963.0, 945.3, 882.2, 802.5, 653.6, 628.8, 478.5, 506.4, 462.8, float('nan')])
N_hat_sd = np.array([float('nan'), 151.2, 129.3, 140.9, 125.5, 96.1, 74.8, 61.7, 61.9, 51.8, 65.8, 70.2, float('nan')])

In [ ]:
p_samples, phi_samples, U_samples, N_samples = \
    unpack_param(samples)

In [ ]:
np.vstack((np.mean(phi_samples, axis=0), phi_hat)).T

In [ ]:
np.vstack((np.std(phi_samples, axis=0), phi_hat_sd)).T

In [ ]:
np.vstack((np.mean(N_samples, 0), N_hat)).T 

In [ ]:
np.vstack((np.std(N_samples, 0), N_hat_sd)).T

In [ ]:
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(p_samples[:1000,:])
plt.subplot(1, 2, 2)
plt.hist(p_samples[:, 0], bins=25)
plt.show()

In [ ]:
plt.plot(U_samples[:1000,:])
plt.show()

### Investigate the correlation structure of the posterior

In [ ]:
param_index = np.hstack(tuple([index["U"][i], index["p"][i]] for i in range(T - 1)))
plt.imshow(np.corrcoef(samples[:, param_index].T), cmap='coolwarm')
plt.clim(-1, 1)
plt.colorbar()
plt.show()

In [ ]:
param_index = np.hstack(tuple([index["U"][i], index["phi"][i]] for i in range(T - 1)))
plt.imshow(np.corrcoef(samples[:, param_index].T), cmap='coolwarm')
plt.clim(-1, 1)
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(np.corrcoef(U_samples.T), cmap='coolwarm')
plt.xlabel('')
plt.clim(-1, 1)
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(7,5), dpi=80)
plt.rcParams['font.size'] = 20
plt.hist2d(logodd(p_samples[:,0]), np.log10(U_samples[:,0]), bins=20, normed=True, cmap='inferno')
plt.xlabel(r'$\log(q_1 / (1 - q_1))$')
plt.ylabel(r'$\log_{10}(N_1)$')
plt.colorbar(ticks=[])
plt.tight_layout()
plt.savefig('jolly_seber_posterior_2dhist.pdf')
plt.show()

In [ ]:
plt.hist2d(np.log10(U_samples[:,2]), logit(phi_samples[:,0]), bins=20, cmap='inferno')
plt.colorbar()
plt.show()

In [ ]:
param_index = np.hstack(tuple([index["phi"][i]] for i in range(T - 1)))
plt.imshow(np.corrcoef(samples[:, param_index].T), cmap='coolwarm')
plt.clim(-1, 1)
plt.colorbar()
plt.show()